In [1]:
#importing the libraies

import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#importing the dataframe
df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
42530,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,NaN,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",film begin wxyz world war ended centre around ...
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...


In [3]:
data=pd.read_csv("most_frequent_words_30.csv")
data

,Genre,Words,Counts
0,western,town,1144
1,western,men,940
2,western,take,683
3,western,back,672
4,western,one,671
...,...,...,...
835,anime,help,95
836,anime,begin,94
837,anime,father,93
838,anime,attack,91


In [4]:
dict_words={i:(data[data['Genre']==i]['Words']).to_list() for i in data['Genre'].unique()}
dict_words


{'western': ['town',
  'men',
  'take',
  'back',
  'one',
  'kill',
  'man',
  'ranch',
  'horse',
  'find',
  'ride',
  'killed',
  'go',
  'gang',
  'return',
  'two',
  'get',
  'gun',
  'sheriff',
  'tell',
  'indian',
  'shoot',
  'father',
  'brother',
  'help',
  'try',
  'woman',
  'gold',
  'new',
  'come'],
 'comedy': ['get',
  'find',
  'go',
  'one',
  'take',
  'tell',
  'friend',
  'back',
  'love',
  'new',
  'two',
  'father',
  'make',
  'life',
  'day',
  'home',
  'time',
  'family',
  'house',
  'also',
  'try',
  'come',
  'meet',
  'end',
  'see',
  'later',
  'man',
  'film',
  'money',
  'however'],
 'short': ['tom',
  'jerry',
  'bug',
  'stooge',
  'get',
  'back',
  'cat',
  'head',
  'spike',
  'curly',
  'mouse',
  'run',
  'go',
  'try',
  'one',
  'house',
  'end',
  'moe',
  'see',
  'away',
  'take',
  'make',
  'door',
  'two',
  'dog',
  'time',
  'chase',
  'fall',
  'sam',
  'find'],
 'action': ['kill',
  'one',
  'police',
  'get',
  'find',
  'ta

In [5]:
def reduce_words(x):
    return ' '.join([i for i in x['Plot_cleanned'].split() if i in  dict_words[x['Genre_updated']]])
     

In [6]:
%%time
Train=df[~pd.isnull(df['Genre_updated'])]
Test=df[pd.isnull(df['Genre_updated'])]
Train['Plot_trimmed']=Train[['Genre_updated','Plot_cleanned']].apply(lambda x: reduce_words(x),axis=1)

Wall time: 1min 21s


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize

In [8]:
Train

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned,Plot_trimmed
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,film open two bandit breaking railroad telegra...,two two kill kill one try horse back tell men ...
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,film family move suburb hoping quiet life thin...,film family life go get
10,1906,Dream of a Rarebit Fiend,American,Wallace McCutcheon and Edwin S. Porter,NaN,short,short,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...,The Rarebit Fiend gorges on Welsh rarebit at a...,rarebit fiend gorge welsh rarebit restaurant l...,get try get fall head back get fall
11,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,action,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,film feature train traveling rockies hold crea...,two later
12,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,western,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,film feature train traveling rockies hold crea...,two horse
...,...,...,...,...,...,...,...,...,...,...,...
42526,2013,Particle (film),Turkish,Erdem Tepegöz,"Jale Arıkan, Rüçhan Caliskur, Özay Fecht, Remz...",drama film,drama,https://en.wikipedia.org/wiki/Particle_(film),"Zeynep lost her job at weaving factory, and he...",zeynep lost job weaving factory family mother ...,family mother find new
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...,two
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...,father also friend try family find
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...,film however two go man go man new life get life


In [9]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(Train['Plot_trimmed'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 178 unique tokens.


In [10]:
X = tokenizer.texts_to_sequences(Train['Plot_trimmed'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (35692, 300)


In [11]:
X_test1 = tokenizer.texts_to_sequences(Test['Plot_cleanned'].values)
X_test1 = pad_sequences(X_test1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test1.shape)

Shape of data tensor: (6843, 300)


In [12]:
Y = pd.get_dummies(Train['Genre_updated']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (35692, 28)


In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(24984, 300) (24984, 28)
(10708, 300) (10708, 28)


In [14]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(28, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          500000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 300, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 28)                2828      
Total params: 583,228
Trainable params: 583,228
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
352/352 [==============================] - 473s 1s/step - loss: 1.8332 - accuracy: 0.4678 - val_loss: 1.1303 - val_accuracy: 0.6731
Epoch 2/5
352/352 [==============================] - 468s 1s/step - loss: 0.9791 - accuracy: 0.7039 - val_loss: 0.8018 - val_accuracy: 0.7515
Epoch 3/5
352/352 [==============================] - 448s 1s/step - loss: 0.7808 - accuracy: 0.7548 - val_loss: 0.6895 - val_accuracy: 0.7839
Epoch 4/5
352/352 [==============================] - 762s 2s/step - loss: 0.6948 - accuracy: 0.7773 - val_loss: 0.6467 - val_accuracy: 0.7923
Epoch 5/5
352/352 [==============================] - 1423s 4s/step - loss: 0.6318 - accuracy: 0.7892 - val_loss: 0.6170 - val_accuracy: 0.7991


In [16]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

335/335 [==============================] - 55s 164ms/step - loss: 0.6151 - accuracy: 0.7972
Test set
  Loss: 0.615
  Accuracy: 0.797


In [17]:
y_pred=model.predict(X_test)

In [18]:
model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([ 2,  1,  8, ...,  5, 17, 13], dtype=int64)

In [43]:
dx=pd.DataFrame(model.predict(X_test1),columns=['action', 'adventure', 'animated', 'anime', 'biography', 'comedy',
       'crime', 'documentary', 'drama', 'family', 'fantasy', 'historical',
       'horror', 'martial_arts', 'musical', 'mystery', 'noir', 'romance',
       'sci_fi', 'serial', 'short', 'social', 'sports', 'spy', 'teen',
       'thriller', 'war', 'western'])

In [44]:
dx

,action,adventure,animated,anime,biography,comedy,crime,documentary,drama,family,...,sci_fi,serial,short,social,sports,spy,teen,thriller,war,western
0,0.000024,0.016313,0.002558,0.023809,0.007732,2.153164e-05,0.000453,0.002172,0.000035,0.000101,...,0.029338,0.335198,0.037813,0.000154,0.205750,0.070560,0.000450,0.000010,0.244093,0.003564
1,0.000354,0.003128,0.039734,0.001035,0.001320,5.319614e-04,0.003880,0.001314,0.000063,0.006263,...,0.001440,0.008522,0.055484,0.008556,0.001522,0.003000,0.002523,0.002755,0.007774,0.007965
2,0.000205,0.001190,0.005183,0.033567,0.381052,9.512302e-04,0.000010,0.327798,0.000401,0.006582,...,0.000978,0.007993,0.018083,0.000590,0.059055,0.005747,0.022554,0.000010,0.005876,0.000897
3,0.000069,0.000504,0.000730,0.007988,0.000633,5.074407e-06,0.000065,0.002356,0.000003,0.000039,...,0.002849,0.662344,0.004571,0.000127,0.186608,0.036643,0.001956,0.000002,0.078665,0.002749
4,0.000023,0.001521,0.000163,0.004459,0.573912,6.816972e-06,0.000016,0.016696,0.000444,0.005423,...,0.002708,0.000730,0.000447,0.001638,0.001731,0.011691,0.001965,0.000014,0.089131,0.000121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6838,0.020509,0.022111,0.000789,0.000344,0.090200,8.528896e-05,0.000076,0.002497,0.059699,0.292747,...,0.000438,0.000052,0.000025,0.059559,0.000141,0.000390,0.000876,0.000282,0.000818,0.000210
6839,0.000092,0.000321,0.198614,0.002386,0.000189,1.212004e-03,0.008764,0.000623,0.000008,0.004023,...,0.001598,0.048237,0.429193,0.002893,0.016031,0.029257,0.031287,0.000975,0.000646,0.000476
6840,0.000089,0.000174,0.025435,0.000955,0.000786,4.497335e-04,0.000011,0.000827,0.000018,0.598195,...,0.000032,0.000059,0.000701,0.009237,0.000139,0.000132,0.302434,0.000094,0.000006,0.000012
6841,0.006774,0.000506,0.006506,0.131115,0.002233,1.483246e-04,0.000109,0.017808,0.000023,0.018906,...,0.002242,0.015295,0.004606,0.016516,0.035275,0.036410,0.443185,0.000313,0.000288,0.000250


In [45]:
def convert(x):
    if x>0.1:
        return 1
    return 0
    
for i in dx.columns:
    dx[i]=dx[i].apply(lambda x:convert(x))

In [46]:
dx.sum(axis=1).value_counts()

2    2512
3    1914
1    1627
4     675
5     109
6       4
0       2
dtype: int64

In [47]:
def genre_pred(col,x):

    if x==1:
        return col

for i in dx.columns:

    dx[i]=dx[i].apply(lambda x:genre_pred(i,x))

In [48]:
dx_list=dx.to_numpy().tolist()

In [49]:
Test['Genre_pred']=dx_list

<ipython-input-49-1e867a2d28ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['Genre_pred']=dx_list


In [50]:
Test['Genre_pred']=Test['Genre_pred'].apply(lambda x:[i for i in x if i])

<ipython-input-50-5d7dce274ac8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['Genre_pred']=Test['Genre_pred'].apply(lambda x:[i for i in x if i])


In [51]:
Test

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned,Genre_pred
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...,"[serial, sports, war]"
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...,"[horror, mystery, noir]"
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...,"[biography, documentary]"
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...,"[serial, sports]"
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...,"[biography, historical]"
...,...,...,...,...,...,...,...,...,...,...,...
42482,2009,Orada,Turkish,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Orada,When the matriarch of an Istanbul family dies ...,matriarch istanbul family dy home elderly son ...,"[family, historical]"
42527,2014,Mandıra Filozofu,Turkish,Director: Müfit Can Saçıntı,Director: Müfit Can Saçıntı\r\nCast: Rasim Özt...,unknown,NaN,https://en.wikipedia.org/wiki/Mand%C4%B1ra_Fil...,Cavit an ambitious industralist in İstanbul pl...,cavit ambitious industralist i̇stanbul plan bu...,"[animated, noir, short]"
42528,2014,Winter Sleep,Turkish,Director: Nuri Bilge Ceylan,Director: Nuri Bilge Ceylan\r\nCast: Haluk Bil...,unknown,NaN,https://en.wikipedia.org/wiki/Winter_Sleep_(film),"Aydın, a former actor, owns a mountaintop hote...",aydın former actor owns mountaintop hotel capp...,"[family, teen]"
42529,2014,Sivas,Turkish,Director: Kaan Müjdeci,Director: Kaan Müjdeci\r\nCast: Dogan Izci,unknown,NaN,https://en.wikipedia.org/wiki/Sivas_(film),The film follows an eleven-year-old boy named ...,film follows elevenyearold boy named aslan dog...,"[anime, fantasy, martial_arts, teen]"


In [52]:
Test.to_csv('test_data_30.csv',index=False)